In [4]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue

import numpy as np
from tqdm.auto import tqdm

In [23]:
PROBLEM_SIZE = 20
NUM_SETS = 50
SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])


def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))


assert goal_check(State(set(range(NUM_SETS)), set())), "Probelm not solvable"

In [6]:
def h(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state, c_set):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in c_set), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

def f(state, c_set):
    return len(state.taken) + h3(state, c_set)

In [24]:
# starting from the solution provided by professor Squillero:
# my idea is to improve the algorithm by reducing the amount of sets to check
# I can do that by removing useless sets such as duplicate sets and the ones made of only "False" attributes
# this should improve both the search for the solution, as well as the calculation for the heuristics
cleaned_set = []
cleaned_range = set(range(NUM_SETS))
i = 0
for s in SETS:
    if sum(s) != 0:
        if (any(np.array_equal(s, cs) for cs in cleaned_set)):
            cleaned_range.remove(i)
        else:
            cleaned_set.append(s)
    else:
        cleaned_range.remove(i)
    i += 1

frontier = PriorityQueue()
state = State(set(), cleaned_range)
frontier.put((f(state, cleaned_set), state))

found


In [20]:
counter = 0

_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state, cleaned_set), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles), ({current_state.taken})")

0it [00:00, ?it/s]

Solved in 119 steps (5 tiles), ({1, 35, 39, 14, 49})
